In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
import tensorflow as tf
import keras

In [3]:
train = pd.read_csv('train.csv')

# Data Preprocessing
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan , strategy='median')
Age_2 = train['Age'].values.reshape(1,-1)
Age_2 = Age_2.T
Age_2 = imputer.fit_transform(Age_2)
train['Age_2'] = Age_2
train.head()

test = pd.read_csv('test.csv')

Age_test = test['Age'].values.reshape(1,-1)
Age_test = Age_test.T
Age_test = imputer.fit_transform(Age_test)
test['Age_2'] = Age_test
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_2
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,34.5
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,47.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,62.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,27.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,22.0


In [4]:
train['Embarked'].value_counts() # As we see that 'S' value is very frequent, we replace the null with 'S'
train.Embarked.fillna('S',inplace = True)

In [5]:
test['Fare'].fillna(test['Fare'].mean(),inplace = True)

In [6]:
train.drop(['Age','Cabin'], inplace = True, axis = 1)
test.drop(['Age','Cabin'], inplace = True, axis = 1)

In [7]:
train = pd.get_dummies(train,columns  =['Sex','Embarked','Pclass'], drop_first = True)
test = pd.get_dummies(test,columns  =['Sex','Embarked','Pclass'], drop_first = True)
train.head()

,PassengerId,Survived,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",1,0,A/5 21171,7.2500,22.0,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0,PC 17599,71.2833,38.0,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",0,0,STON/O2. 3101282,7.9250,26.0,0,0,1,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0,113803,53.1000,35.0,0,0,1,0,0
4,5,0,"Allen, Mr. William Henry",0,0,373450,8.0500,35.0,1,0,1,0,1


In [8]:
train_target = train.iloc[:,1].values
train_predictors = train.iloc[:,6:].values
test_predictors = test.iloc[:,5:].values


#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_predictors = sc.fit_transform(train_predictors)
test_predictors = sc.transform(test_predictors)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train_predictors, train_target, test_size = 0.2, random_state = 0)
n_cols = train_predictors.shape[1]
n_cols

7

In [35]:
def build_model(hp):
    
    model = keras.Sequential([keras.layers.Dense(
            units = hp.Int('Dense_1_unit', min_value = 100, max_value = 150, step = 10),
            activation = tf.keras.layers.LeakyReLU(alpha= 0.3),
            input_shape = (n_cols,)
        ),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(
            units = hp.Int('Dense_1_unit', min_value = 50, max_value = 100, step = 10),
            activation = tf.keras.layers.LeakyReLU(alpha= 0.3)
        ),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(
            units = hp.Int('Dense_3_unit', min_value = 25, max_value = 50),
            activation = tf.keras.layers.LeakyReLU(alpha= 0.3)
        ),
        keras.layers.Dense(1, activation = 'sigmoid')
    ])

    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2,1e-3])),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
    
    return model

In [36]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [37]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, project_name = 'hh_1')

In [38]:
tuner_search.search(train_predictors, train_target, epochs = 3, validation_split = 0.1)

Epoch 1/3
26/26 [==============================] - ETA: 0s - loss: 0.6851 - accuracy: 0.59 - ETA: 0s - loss: 0.5335 - accuracy: 0.76 - 1s 21ms/step - loss: 0.5238 - accuracy: 0.7690 - val_loss: 0.4995 - val_accuracy: 0.8111
Epoch 2/3
26/26 [==============================] - ETA: 0s - loss: 0.2751 - accuracy: 0.93 - ETA: 0s - loss: 0.5228 - accuracy: 0.77 - 0s 4ms/step - loss: 0.5101 - accuracy: 0.7765 - val_loss: 0.4449 - val_accuracy: 0.7778
Epoch 3/3
26/26 [==============================] - ETA: 0s - loss: 0.3752 - accuracy: 0.78 - ETA: 0s - loss: 0.4874 - accuracy: 0.79 - 0s 12ms/step - loss: 0.4693 - accuracy: 0.7978 - val_loss: 0.4189 - val_accuracy: 0.8222


Epoch 1/3
26/26 [==============================] - ETA: 0s - loss: 0.7069 - accuracy: 0.43 - ETA: 0s - loss: 0.5481 - accuracy: 0.71 - 1s 26ms/step - loss: 0.5535 - accuracy: 0.7353 - val_loss: 0.4600 - val_accuracy: 0.8333
Epoch 2/3
26/26 [==============================] - ETA: 0s - loss: 0.4974 - accuracy: 0.75 - ETA: 0s - loss: 0.4855 - accuracy: 0.77 - 0s 5ms/step - loss: 0.4889 - accuracy: 0.7753 - val_loss: 0.4411 - val_accuracy: 0.7889
Epoch 3/3
26/26 [==============================] - ETA: 0s - loss: 0.7194 - accuracy: 0.71 - ETA: 0s - loss: 0.4864 - accuracy: 0.78 - 0s 5ms/step - loss: 0.4756 - accuracy: 0.7940 - val_loss: 0.4265 - val_accuracy: 0.8000


Epoch 1/3
26/26 [==============================] - ETA: 0s - loss: 0.7092 - accuracy: 0.59 - ETA: 0s - loss: 0.5519 - accuracy: 0.74 - 0s 19ms/step - loss: 0.5385 - accuracy: 0.7478 - val_loss: 0.4182 - val_accuracy: 0.8111
Epoch 2/3
26/26 [==============================] - ETA: 0s - loss: 0.5328 - accuracy: 0.68 - ETA: 0s - loss: 0.4629 - accuracy: 0.77 - 0s 5ms/step - loss: 0.5028 - accuracy: 0.7628 - val_loss: 0.4551 - val_accuracy: 0.7778
Epoch 3/3
26/26 [==============================] - ETA: 0s - loss: 0.5107 - accuracy: 0.71 - ETA: 0s - loss: 0.4795 - accuracy: 0.77 - 0s 14ms/step - loss: 0.4811 - accuracy: 0.7853 - val_loss: 0.4185 - val_accuracy: 0.8556


Epoch 1/3
26/26 [==============================] - ETA: 0s - loss: 0.8392 - accuracy: 0.40 - ETA: 0s - loss: 0.5501 - accuracy: 0.72 - 1s 20ms/step - loss: 0.5379 - accuracy: 0.7416 - val_loss: 0.4606 - val_accuracy: 0.8111
Epoch 2/3
26/26 [==============================] - ETA: 0s - loss: 0.4430 - accuracy: 0.84 - ETA: 0s - loss: 0.5019 - accuracy: 0.78 - 0s 13ms/step - loss: 0.5008 - accuracy: 0.7853 - val_loss: 0.4353 - val_accuracy: 0.8222
Epoch 3/3
26/26 [==============================] - ETA: 0s - loss: 0.4413 - accuracy: 0.78 - ETA: 0s - loss: 0.4842 - accuracy: 0.79 - 0s 12ms/step - loss: 0.4657 - accuracy: 0.8052 - val_loss: 0.4165 - val_accuracy: 0.8333


Epoch 1/3
26/26 [==============================] - ETA: 0s - loss: 0.7100 - accuracy: 0.59 - ETA: 0s - loss: 0.6000 - accuracy: 0.71 - 1s 24ms/step - loss: 0.5872 - accuracy: 0.7191 - val_loss: 0.4698 - val_accuracy: 0.7889
Epoch 2/3
26/26 [==============================] - ETA: 0s - loss: 0.4741 - accuracy: 0.81 - ETA: 0s - loss: 0.4911 - accuracy: 0.77 - 0s 5ms/step - loss: 0.4728 - accuracy: 0.7903 - val_loss: 0.4536 - val_accuracy: 0.7778
Epoch 3/3
26/26 [==============================] - ETA: 0s - loss: 0.4322 - accuracy: 0.81 - ETA: 0s - loss: 0.4848 - accuracy: 0.78 - 0s 7ms/step - loss: 0.4896 - accuracy: 0.7878 - val_loss: 0.4449 - val_accuracy: 0.7778


INFO:tensorflow:Oracle triggered exit


In [39]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               960       
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               14520     
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 29)                3509      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 30        
Total params: 19,019
Trainable params: 19,019
Non-trainable params: 0
____________________________________________________

In [41]:
model.fit(train_predictors,train_target,epochs = 100, validation_split = 0.1, initial_epoch = 3)

Epoch 4/100
26/26 [==============================] - ETA: 0s - loss: 0.3272 - accuracy: 0.84 - ETA: 0s - loss: 0.4716 - accuracy: 0.78 - 0s 9ms/step - loss: 0.4661 - accuracy: 0.7853 - val_loss: 0.4250 - val_accuracy: 0.8333
Epoch 5/100
26/26 [==============================] - ETA: 0s - loss: 0.3217 - accuracy: 0.81 - ETA: 0s - loss: 0.4306 - accuracy: 0.80 - 0s 5ms/step - loss: 0.4555 - accuracy: 0.7878 - val_loss: 0.4290 - val_accuracy: 0.8333
Epoch 6/100
26/26 [==============================] - ETA: 0s - loss: 0.6261 - accuracy: 0.68 - ETA: 0s - loss: 0.5093 - accuracy: 0.77 - 0s 5ms/step - loss: 0.4797 - accuracy: 0.7878 - val_loss: 0.4399 - val_accuracy: 0.8111
Epoch 7/100
26/26 [==============================] - ETA: 0s - loss: 0.2737 - accuracy: 0.87 - ETA: 0s - loss: 0.4873 - accuracy: 0.79 - 0s 5ms/step - loss: 0.4691 - accuracy: 0.8052 - val_loss: 0.4295 - val_accuracy: 0.8000
Epoch 8/100
26/26 [==============================] - ETA: 0s - loss: 0.3324 - accuracy: 0.90 - ETA: 

Epoch 40/100
26/26 [==============================] - ETA: 0s - loss: 0.4933 - accuracy: 0.68 - ETA: 0s - loss: 0.4338 - accuracy: 0.78 - 0s 5ms/step - loss: 0.4452 - accuracy: 0.7965 - val_loss: 0.4062 - val_accuracy: 0.8333
Epoch 41/100
26/26 [==============================] - ETA: 0s - loss: 0.4288 - accuracy: 0.81 - ETA: 0s - loss: 0.4490 - accuracy: 0.82 - 0s 4ms/step - loss: 0.4508 - accuracy: 0.8165 - val_loss: 0.4009 - val_accuracy: 0.8333
Epoch 42/100
26/26 [==============================] - ETA: 0s - loss: 0.4665 - accuracy: 0.75 - ETA: 0s - loss: 0.4324 - accuracy: 0.80 - 0s 4ms/step - loss: 0.4377 - accuracy: 0.8065 - val_loss: 0.4174 - val_accuracy: 0.8333
Epoch 43/100
26/26 [==============================] - ETA: 0s - loss: 0.4799 - accuracy: 0.71 - ETA: 0s - loss: 0.4895 - accuracy: 0.77 - 0s 4ms/step - loss: 0.4713 - accuracy: 0.7903 - val_loss: 0.4148 - val_accuracy: 0.8222
Epoch 44/100
26/26 [==============================] - ETA: 0s - loss: 0.3845 - accuracy: 0.84 - 

26/26 [==============================] - ETA: 0s - loss: 0.5245 - accuracy: 0.68 - ETA: 0s - loss: 0.4329 - accuracy: 0.81 - 0s 5ms/step - loss: 0.4555 - accuracy: 0.8027 - val_loss: 0.4183 - val_accuracy: 0.8222
Epoch 77/100
26/26 [==============================] - ETA: 0s - loss: 0.4841 - accuracy: 0.71 - ETA: 0s - loss: 0.4385 - accuracy: 0.81 - 0s 4ms/step - loss: 0.4511 - accuracy: 0.8015 - val_loss: 0.4011 - val_accuracy: 0.8333
Epoch 78/100
26/26 [==============================] - ETA: 0s - loss: 0.3305 - accuracy: 0.84 - ETA: 0s - loss: 0.4464 - accuracy: 0.80 - 0s 4ms/step - loss: 0.4428 - accuracy: 0.8102 - val_loss: 0.4110 - val_accuracy: 0.8222
Epoch 79/100
26/26 [==============================] - ETA: 0s - loss: 0.5234 - accuracy: 0.75 - ETA: 0s - loss: 0.4934 - accuracy: 0.78 - 0s 4ms/step - loss: 0.4790 - accuracy: 0.7903 - val_loss: 0.4396 - val_accuracy: 0.8111
Epoch 80/100
26/26 [==============================] - ETA: 0s - loss: 0.4431 - accuracy: 0.81 - ETA: 0s - los

In [42]:
y_pred = model.predict(X_test).round()

In [43]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[102   8]
 [ 26  43]]


In [44]:
survived = model.predict(test_predictors).round()

In [45]:
test['Survived'] = survived
test['Survived'] = test['Survived'].astype(int)

In [46]:
test.head()

,PassengerId,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Survived
0,892,"Kelly, Mr. James",0,0,330911,7.8292,34.5,1,1,0,0,1,0
1,893,"Wilkes, Mrs. James (Ellen Needs)",1,0,363272,7.0000,47.0,0,0,1,0,1,0
2,894,"Myles, Mr. Thomas Francis",0,0,240276,9.6875,62.0,1,1,0,1,0,0
3,895,"Wirz, Mr. Albert",0,0,315154,8.6625,27.0,1,0,1,0,1,0
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1,3101298,12.2875,22.0,0,0,1,0,1,0


In [47]:
Final = test.drop(test.columns.difference(['PassengerId','Survived']), axis = 1)

In [48]:
Final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [49]:
Final.to_csv('Titanic_with DeepLearning_5(keras_tuner).csv', index=False)